In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from geopy.distance import great_circle

### Take the sample of 1000 taxis

In [8]:
taxis_numbers = 1000
#time_interval = "6H"  # Pandas uses uppercase 'H' for hours
DATASET_FOLDER = "T-Drive_Taxi_Trajectory/taxi_log_2008_by_id"

df = pd.DataFrame()
# Loop through taxi files and concatenate data
for i in range(1, taxis_numbers + 1):
    df_taxi = pd.read_csv(
        f"{DATASET_FOLDER}/{i}.txt",
        header=None,
        names=['id', 'datetime', 'longitude', 'latitude']
    )
    df = pd.concat([df, df_taxi], ignore_index=True)

# Convert 'datetime' column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")

# Create a 'coordinates' column as tuples of (longitude, latitude)
df['coordinates'] = list(zip(df['longitude'], df['latitude']))
# Sort by 'datetime'
df = df.sort_values(by='datetime')
df = df.set_index('datetime')

In [9]:
df

,id,longitude,latitude,coordinates
datetime,,,,
2008-02-02 13:30:47,903,116.20268,39.99162,"(116.20268, 39.99162)"
2008-02-02 13:30:48,366,116.45353,39.90732,"(116.45353, 39.90732)"
2008-02-02 13:30:49,950,116.29443,39.88372,"(116.29443, 39.88372)"
2008-02-02 13:30:51,875,116.30687,40.02802,"(116.30687, 40.02802)"
2008-02-02 13:30:52,719,116.34647,40.00016,"(116.34647, 40.00016)"
...,...,...,...,...
2008-02-08 17:39:17,149,116.41811,39.97870,"(116.41811, 39.9787)"
2008-02-08 17:39:18,654,116.41112,39.85563,"(116.41112, 39.85563)"
2008-02-08 17:39:18,417,116.42741,39.93292,"(116.42741, 39.93292)"


# Create traffic using different ways

### How many cars are there on the road at a given time (15 minute intervals)?

In [22]:
unique_cars_per_interval = df.resample('15T')['id'].nunique().reset_index()
unique_cars_per_interval.columns = ['datetime', 'unique_cars']
unique_cars_per_interval

,datetime,unique_cars
0,2008-02-02 13:30:00,738
1,2008-02-02 13:45:00,778
2,2008-02-02 14:00:00,788
3,2008-02-02 14:15:00,782
4,2008-02-02 14:30:00,770
...,...,...
588,2008-02-08 16:30:00,661
589,2008-02-08 16:45:00,666
590,2008-02-08 17:00:00,673
591,2008-02-08 17:15:00,671


In [23]:
unique_cars_per_interval.unique_cars.describe()

count    593.000000
mean     627.630691
std      117.879019
min        0.000000
25%      520.000000
50%      634.000000
75%      738.000000
max      812.000000
Name: unique_cars, dtype: float64

So we see there is a difference

In [24]:
unique_cars_per_interval.sort_values(by='unique_cars', ascending=False).head(10)

,datetime,unique_cars
94,2008-02-03 13:00:00,812
93,2008-02-03 12:45:00,811
186,2008-02-04 12:00:00,808
188,2008-02-04 12:30:00,805
17,2008-02-02 17:45:00,803
96,2008-02-03 13:30:00,801
18,2008-02-02 18:00:00,801
19,2008-02-02 18:15:00,801
95,2008-02-03 13:15:00,800
189,2008-02-04 12:45:00,800


In [25]:
unique_cars_per_interval.sort_values(by='unique_cars', ascending=True).head(10)

,datetime,unique_cars
532,2008-02-08 02:30:00,0
243,2008-02-05 02:15:00,37
530,2008-02-08 02:00:00,160
434,2008-02-07 02:00:00,415
531,2008-02-08 02:15:00,444
448,2008-02-07 05:30:00,453
545,2008-02-08 05:45:00,457
456,2008-02-07 07:30:00,459
549,2008-02-08 06:45:00,459
543,2008-02-08 05:15:00,459


### What is the percentage change between the current time period and the previous time period?

In [26]:
unique_cars_per_interval['l1'] = unique_cars_per_interval['unique_cars'].shift(1)
unique_cars_per_interval['percent_change'] = (
    (unique_cars_per_interval['unique_cars'] - unique_cars_per_interval['l1'])
    / unique_cars_per_interval['l1']
) * 100
unique_cars_per_interval

,datetime,unique_cars,l1,percent_change
0,2008-02-02 13:30:00,738,NaN,NaN
1,2008-02-02 13:45:00,778,738.0,5.420054
2,2008-02-02 14:00:00,788,778.0,1.285347
3,2008-02-02 14:15:00,782,788.0,-0.761421
4,2008-02-02 14:30:00,770,782.0,-1.534527
...,...,...,...,...
588,2008-02-08 16:30:00,661,664.0,-0.451807
589,2008-02-08 16:45:00,666,661.0,0.756430
590,2008-02-08 17:00:00,673,666.0,1.051051
591,2008-02-08 17:15:00,671,673.0,-0.297177


In [27]:
unique_cars_per_interval['percent_change'].describe()

count    592.000000
mean            inf
std             NaN
min     -100.000000
25%       -1.258357
50%       -0.127476
75%        1.188282
max             inf
Name: percent_change, dtype: float64

**So either unique_cars or percent_change can be used to measure the traffic (the more - the heavier the traffic is)**

**Maybe it is better to look at sum of the percentage changes (say for 5-10 observations), because:**

+ If the traffic was growing for 5 times in a row, it means that the traffic is very heavy

+ Next, the traffic started to decrease, but for some time is is still heavy

+ Just percentage change won't show it, but the cumulative sum will

In [28]:
# Calculate the rolling sum of the 'percent_change' column over the current row and the previous 4 rows
unique_cars_per_interval['cumulative_percent_change'] = (
    unique_cars_per_interval['percent_change']
    .rolling(window=5)
    .sum()
)

unique_cars_per_interval

,datetime,unique_cars,l1,percent_change,cumulative_percent_change
0,2008-02-02 13:30:00,738,NaN,NaN,NaN
1,2008-02-02 13:45:00,778,738.0,5.420054,NaN
2,2008-02-02 14:00:00,788,778.0,1.285347,NaN
3,2008-02-02 14:15:00,782,788.0,-0.761421,NaN
4,2008-02-02 14:30:00,770,782.0,-1.534527,NaN
...,...,...,...,...,...
588,2008-02-08 16:30:00,661,664.0,-0.451807,3.094010
589,2008-02-08 16:45:00,666,661.0,0.756430,3.694433
590,2008-02-08 17:00:00,673,666.0,1.051051,3.655141
591,2008-02-08 17:15:00,671,673.0,-0.297177,2.587549


In [30]:
unique_cars_per_interval.head(20)

,datetime,unique_cars,l1,percent_change,cumulative_percent_change
0,2008-02-02 13:30:00,738,NaN,NaN,NaN
1,2008-02-02 13:45:00,778,738.0,5.420054,NaN
2,2008-02-02 14:00:00,788,778.0,1.285347,NaN
3,2008-02-02 14:15:00,782,788.0,-0.761421,NaN
4,2008-02-02 14:30:00,770,782.0,-1.534527,NaN
5,2008-02-02 14:45:00,761,770.0,-1.168831,3.240622
6,2008-02-02 15:00:00,763,761.0,0.262812,-1.916620
7,2008-02-02 15:15:00,748,763.0,-1.965924,-5.167891
8,2008-02-02 15:30:00,753,748.0,0.668449,-3.738021
9,2008-02-02 15:45:00,764,753.0,1.460823,-0.742670


In [35]:
unique_cars_per_interval['percent_change'][5] + \
unique_cars_per_interval['percent_change'][4] + \
unique_cars_per_interval['percent_change'][3] + \
unique_cars_per_interval['percent_change'][2] + \
unique_cars_per_interval['percent_change'][1]

3.240621901395733

In [36]:
unique_cars_per_interval['percent_change'][7] + \
unique_cars_per_interval['percent_change'][6] + \
unique_cars_per_interval['percent_change'][5] + \
unique_cars_per_interval['percent_change'][4] + \
unique_cars_per_interval['percent_change'][3]

-5.16789123776457

**Divide the current unique_cars by the l4 to check the total changes within more time**

In [38]:
unique_cars_per_interval['l4'] = unique_cars_per_interval['unique_cars'].shift(4)
unique_cars_per_interval['percent_change_4'] = (
    (unique_cars_per_interval['unique_cars'] - unique_cars_per_interval['l4'])
    / unique_cars_per_interval['l4']
) * 100
unique_cars_per_interval

,datetime,unique_cars,l1,percent_change,cumulative_percent_change,l4,percent_change_4
0,2008-02-02 13:30:00,738,NaN,NaN,NaN,NaN,NaN
1,2008-02-02 13:45:00,778,738.0,5.420054,NaN,NaN,NaN
2,2008-02-02 14:00:00,788,778.0,1.285347,NaN,NaN,NaN
3,2008-02-02 14:15:00,782,788.0,-0.761421,NaN,NaN,NaN
4,2008-02-02 14:30:00,770,782.0,-1.534527,NaN,738.0,4.336043
...,...,...,...,...,...,...,...
588,2008-02-08 16:30:00,661,664.0,-0.451807,3.094010,642.0,2.959502
589,2008-02-08 16:45:00,666,661.0,0.756430,3.694433,649.0,2.619414
590,2008-02-08 17:00:00,673,666.0,1.051051,3.655141,654.0,2.905199
591,2008-02-08 17:15:00,671,673.0,-0.297177,2.587549,664.0,1.054217


In [39]:
unique_cars_per_interval['percent_change_4'].describe()

count    589.000000
mean            inf
std             NaN
min     -100.000000
25%       -3.651685
50%       -0.772798
75%        2.474527
max             inf
Name: percent_change_4, dtype: float64

### What if we change the time interval to 1 hour?

In [18]:
unique_cars_per_interval = df.resample('1H')['id'].nunique().reset_index()
unique_cars_per_interval.columns = ['datetime', 'unique_cars']
unique_cars_per_interval

,datetime,unique_cars
0,2008-02-02 13:00:00,808
1,2008-02-02 14:00:00,841
2,2008-02-02 15:00:00,832
3,2008-02-02 16:00:00,841
4,2008-02-02 17:00:00,854
...,...,...
144,2008-02-08 13:00:00,767
145,2008-02-08 14:00:00,712
146,2008-02-08 15:00:00,715
147,2008-02-08 16:00:00,728


In [19]:
unique_cars_per_interval['l1'] = unique_cars_per_interval['unique_cars'].shift(1)
unique_cars_per_interval['percent_change'] = (
    (unique_cars_per_interval['unique_cars'] - unique_cars_per_interval['l1'])
    / unique_cars_per_interval['l1']
) * 100
unique_cars_per_interval

,datetime,unique_cars,l1,percent_change
0,2008-02-02 13:00:00,808,NaN,NaN
1,2008-02-02 14:00:00,841,808.0,4.084158
2,2008-02-02 15:00:00,832,841.0,-1.070155
3,2008-02-02 16:00:00,841,832.0,1.081731
4,2008-02-02 17:00:00,854,841.0,1.545779
...,...,...,...,...
144,2008-02-08 13:00:00,767,803.0,-4.483188
145,2008-02-08 14:00:00,712,767.0,-7.170795
146,2008-02-08 15:00:00,715,712.0,0.421348
147,2008-02-08 16:00:00,728,715.0,1.818182


In [20]:
unique_cars_per_interval['percent_change'].describe()

count    148.000000
mean       0.094318
std        6.056215
min      -12.149533
25%       -4.100451
50%       -1.130315
75%        1.859299
max       20.458554
Name: percent_change, dtype: float64

### More smart approach - split the locations into several zones (like districts) based on the coordinates, and do the exact same thing

In [41]:
df

,id,longitude,latitude,coordinates
datetime,,,,
2008-02-02 13:30:47,903,116.20268,39.99162,"(116.20268, 39.99162)"
2008-02-02 13:30:48,366,116.45353,39.90732,"(116.45353, 39.90732)"
2008-02-02 13:30:49,950,116.29443,39.88372,"(116.29443, 39.88372)"
2008-02-02 13:30:51,875,116.30687,40.02802,"(116.30687, 40.02802)"
2008-02-02 13:30:52,719,116.34647,40.00016,"(116.34647, 40.00016)"
...,...,...,...,...
2008-02-08 17:39:17,149,116.41811,39.97870,"(116.41811, 39.9787)"
2008-02-08 17:39:18,654,116.41112,39.85563,"(116.41112, 39.85563)"
2008-02-08 17:39:18,417,116.42741,39.93292,"(116.42741, 39.93292)"


**Use other sources**

In [ ]:
# BUT FIRST NEED TO FIGURE OUT WHERE TO GET THIS SHP FILE WITH DISTRICTS

import geopandas as gpd
from shapely.geometry import Point

# Load shapefile of Beijing districts
districts = gpd.read_file("beijing_districts.shp")
# Convert coordinates to Points
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
# Spatial join to assign districts
gdf_with_districts = gpd.sjoin(gdf, districts, how="left", op="within")

**Do it by ourselves using Grid-Based Partitioning**

In [52]:
# interesting, let's filter these weird observations
df.longitude.min(), df.longitude.max(), df.latitude.min(), df.latitude.max()

(0.0, 195.01217, 0.0, 72.08014)

In [ ]:
pd.options.display.float_format = '{:.6f}'.format
df.longitude.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])
# filtering between 115.9 and 117.2 is fine

count   1571163.000000
mean        116.376141
std           2.678331
min           0.000000
1%          115.959990
10%         116.281790
25%         116.344570
50%         116.417290
75%         116.473820
90%         116.651320
99%         117.122544
max         195.012170
Name: longitude, dtype: float64

In [ ]:
df.latitude.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])
# between 39.5 and 40.5 is fine

count   1571163.000000
mean         39.901802
std           1.000450
min           0.000000
1%           39.591100
10%          39.806220
25%          39.871280
50%          39.917360
75%          39.976810
90%          40.103730
99%          40.403930
max          72.080140
Name: latitude, dtype: float64

In [58]:
# filter out the weird observations
df = df[(df.longitude > 115.9) & (df.longitude < 117.2)]
df = df[(df.latitude > 39.5) & (df.latitude < 40.5)]
df

,id,longitude,latitude,coordinates
datetime,,,,
2008-02-02 13:30:47,903,116.202680,39.991620,"(116.20268, 39.99162)"
2008-02-02 13:30:48,366,116.453530,39.907320,"(116.45353, 39.90732)"
2008-02-02 13:30:49,950,116.294430,39.883720,"(116.29443, 39.88372)"
2008-02-02 13:30:51,875,116.306870,40.028020,"(116.30687, 40.02802)"
2008-02-02 13:30:52,719,116.346470,40.000160,"(116.34647, 40.00016)"
...,...,...,...,...
2008-02-08 17:39:17,366,116.449450,39.806750,"(116.44945, 39.80675)"
2008-02-08 17:39:17,149,116.418110,39.978700,"(116.41811, 39.9787)"
2008-02-08 17:39:18,654,116.411120,39.855630,"(116.41112, 39.85563)"


**So we see there were some errors and also some taxis were outside Beijing, but we filtered it**

In [59]:
min_lon, max_lon = df.longitude.min(), df.longitude.max()
min_lat, max_lat = df.latitude.min(), df.latitude.max()
grid_size = 0.01  # ~1km resolution
# Create grid cells
lon_bins = np.arange(min_lon, max_lon, grid_size)
lat_bins = np.arange(min_lat, max_lat, grid_size)
df['grid_id'] = df.apply(lambda row: f"{np.digitize(row.longitude, lon_bins)}_{np.digitize(row.latitude, lat_bins)}", axis=1)
df

,id,longitude,latitude,coordinates,grid_id
datetime,,,,,
2008-02-02 13:30:47,903,116.202680,39.991620,"(116.20268, 39.99162)",31_50
2008-02-02 13:30:48,366,116.453530,39.907320,"(116.45353, 39.90732)",56_41
2008-02-02 13:30:49,950,116.294430,39.883720,"(116.29443, 39.88372)",40_39
2008-02-02 13:30:51,875,116.306870,40.028020,"(116.30687, 40.02802)",41_53
2008-02-02 13:30:52,719,116.346470,40.000160,"(116.34647, 40.00016)",45_50
...,...,...,...,...,...
2008-02-08 17:39:17,366,116.449450,39.806750,"(116.44945, 39.80675)",55_31
2008-02-08 17:39:17,149,116.418110,39.978700,"(116.41811, 39.9787)",52_48
2008-02-08 17:39:18,654,116.411120,39.855630,"(116.41112, 39.85563)",52_36


In [60]:
df['grid_id'].nunique()

5576

**Too many, let's increase the radius to 5 km**

In [61]:
min_lon, max_lon = df.longitude.min(), df.longitude.max()
min_lat, max_lat = df.latitude.min(), df.latitude.max()
grid_size = 0.05  # ~1km resolution
# Create grid cells
lon_bins = np.arange(min_lon, max_lon, grid_size)
lat_bins = np.arange(min_lat, max_lat, grid_size)
df['grid_id'] = df.apply(lambda row: f"{np.digitize(row.longitude, lon_bins)}_{np.digitize(row.latitude, lat_bins)}", axis=1)
df

,id,longitude,latitude,coordinates,grid_id
datetime,,,,,
2008-02-02 13:30:47,903,116.202680,39.991620,"(116.20268, 39.99162)",7_10
2008-02-02 13:30:48,366,116.453530,39.907320,"(116.45353, 39.90732)",12_9
2008-02-02 13:30:49,950,116.294430,39.883720,"(116.29443, 39.88372)",8_8
2008-02-02 13:30:51,875,116.306870,40.028020,"(116.30687, 40.02802)",9_11
2008-02-02 13:30:52,719,116.346470,40.000160,"(116.34647, 40.00016)",9_10
...,...,...,...,...,...
2008-02-08 17:39:17,366,116.449450,39.806750,"(116.44945, 39.80675)",11_7
2008-02-08 17:39:17,149,116.418110,39.978700,"(116.41811, 39.9787)",11_10
2008-02-08 17:39:18,654,116.411120,39.855630,"(116.41112, 39.85563)",11_8


In [ ]:
# much better
df['grid_id'].nunique()

455

In [64]:
df = df.reset_index()
df['datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df

,datetime,id,longitude,latitude,coordinates,grid_id
0,2008-02-02 13:30:47,903,116.202680,39.991620,"(116.20268, 39.99162)",7_10
1,2008-02-02 13:30:48,366,116.453530,39.907320,"(116.45353, 39.90732)",12_9
2,2008-02-02 13:30:49,950,116.294430,39.883720,"(116.29443, 39.88372)",8_8
3,2008-02-02 13:30:51,875,116.306870,40.028020,"(116.30687, 40.02802)",9_11
4,2008-02-02 13:30:52,719,116.346470,40.000160,"(116.34647, 40.00016)",9_10
...,...,...,...,...,...,...
1547280,2008-02-08 17:39:17,366,116.449450,39.806750,"(116.44945, 39.80675)",11_7
1547281,2008-02-08 17:39:17,149,116.418110,39.978700,"(116.41811, 39.9787)",11_10
1547282,2008-02-08 17:39:18,654,116.411120,39.855630,"(116.41112, 39.85563)",11_8
1547283,2008-02-08 17:39:18,417,116.427410,39.932920,"(116.42741, 39.93292)",11_9


In [65]:
# Create a new DataFrame with the grid_id and the number of unique taxis in each grid cell (for every hour)

unique_cars_per_grid = (
    df.groupby([pd.Grouper(key='datetime', freq='1H'), 'grid_id'])['id']
    .nunique()
    .reset_index()
)
# Rename columns for clarity
unique_cars_per_grid.columns = ['datetime', 'grid_id', 'unique_cars']
unique_cars_per_grid

,datetime,grid_id,unique_cars
0,2008-02-02 13:00:00,10_10,92
1,2008-02-02 13:00:00,10_11,10
2,2008-02-02 13:00:00,10_12,1
3,2008-02-02 13:00:00,10_3,1
4,2008-02-02 13:00:00,10_7,20
...,...,...,...
26478,2008-02-08 17:00:00,9_5,10
26479,2008-02-08 17:00:00,9_6,6
26480,2008-02-08 17:00:00,9_7,17
26481,2008-02-08 17:00:00,9_8,66


In [67]:
unique_cars_per_grid.loc[unique_cars_per_grid['grid_id'] == '10_10']

,datetime,grid_id,unique_cars
0,2008-02-02 13:00:00,10_10,92
104,2008-02-02 14:00:00,10_10,138
236,2008-02-02 15:00:00,10_10,132
380,2008-02-02 16:00:00,10_10,124
513,2008-02-02 17:00:00,10_10,134
...,...,...,...
25353,2008-02-08 13:00:00,10_10,72
25586,2008-02-08 14:00:00,10_10,76
25811,2008-02-08 15:00:00,10_10,75
26025,2008-02-08 16:00:00,10_10,73


# Okay so the dataset works. Now we need to answer the following questions:

+ How exactly we define the traffic, and how we will calculate it?

+ How we separate the coordinates into different zones? (like districts)

+ How often do we need to recalculate the traffic? (like every hour, every 30 mins)?

+ How strongly we will filter the data? Need to set the min latitute and longitude, and max latitude and longitude, filtering the observations that do not fit

**After we answer these questions, we can prepare the full version of the dataset**